[DEPRECATED]

The results are outdated and the methods here are no longer need. Leaving for reference

In [ ]:
import csv
import numpy as np
import os
import random as rd
import re
import pandas as pd
import seaborn as sns
import scipy.stats as stats

In [ ]:
%pip install pingouin 
from pingouin import ancova
from pingouin import anova

In [ ]:
os.chdir('/data/Reback-DTI/')

In [ ]:
outcomes = ['pcpcgp_6mo', 'mortality_6mos']

tracts = ['CNGLM_PRHP_L','CNGLM_PRHP_R', 'CST_L','CST_R', 'IFOF_L','IFOF_R', 'IFL_L','IFL_R', 'CCBody']
measures = ['fa', 'rd', 'md', 'ad']

agerange = ['all_ages','ages0_1','ages1_5', 'ages0_5']
selected_agerange = agerange[0]

datasrc = '/data/Reback-DTI/resources/strats/Eddy_corrected/nonadjusted/{}/{}.csv'.format(selected_agerange,selected_agerange)

outputBaseDir = '/data/Reback-DTI/resources/result_stats/{}'.format(selected_agerange)
anovaDir = os.path.join(outputBaseDir, 'anova')
if not os.path.exists(anovaDir):
    os.makedirs(anovaDir)
ttestDir = os.path.join(outputBaseDir, 'ttest')
if not os.path.exists(ttestDir):
    os.makedirs(ttestDir)

In [ ]:
def getData(src):
    pdata = pd.read_csv(src)
    pdata['id'] = pdata['StudyID'].map(str) + '_' + pdata['MR_num'].map(str) + '_' + pdata['Scan_num'].map(str)
    #place_holder for future, won't currently delete anything probably
    pdata = pdata.drop_duplicates(subset='id')  
    pdata.head()
    
    #To make plot labels fit better
    pdata.columns = pdata.columns.str.replace(r"Cingulum_Parahippocampal", "CNGLM_PRHP")
    pdata.columns = pdata.columns.str.replace(r"Corpus_Callosum_Body", "CCBody")
    pdata.columns = pdata.columns.str.replace(r"Corticospinal_Tract", "CST")
    pdata.columns = pdata.columns.str.replace(r"Inferior_Fronto_Occipital_Fasciculus", "IFOF")
    pdata.columns = pdata.columns.str.replace(r"Inferior_Longitudinal_Fasciculus", "IFL")
    return pdata


def getTractMeasure(pdata, outcome=outcomes[1],tract='', measure='fa'): #default is all tracts FA
    tract_measure= re.compile(r''+tract+'.{}$'.format(measure))
    filtered = list(filter(tract_measure.search, pdata.columns))
    #print(filtered)

    ### USE MELT TO CONVERT FROM WIDE TO LONG FORM
    df_meas = pd.melt(pdata, id_vars=['id', outcome, 'AgeGroup', 'Sex'], value_vars=filtered, var_name='Tract', value_name='{}'.format(measure.upper())) 
    df_meas['Tract'] = df_meas['Tract'].str.strip('.{}'.format(measure))
    df_meas = df_meas.dropna()
    df_meas.head()
    return df_meas 



The following is to get ANOVA tables

In [ ]:
def getANOVA(datapd, measure=measures[0], outcome=outcomes[0]):
    filename = "{}_{}_agesexadjusted.txt".format(measure, outcome)
    filepath = os.path.join(anovaDir, filename)
    f = open(filepath, "w")
    for i in range(len(tracts)):
        tract_measures = getTractMeasure(datapd, outcome, tracts[i], measure)
        ancova_age = ancova(data=tract_measures, dv='{}'.format(measure.upper()), covar=['AgeGroup'], between=outcome)
        ancova_agesex = ancova(data=tract_measures, dv='{}'.format(measure.upper()), covar=['AgeGroup', 'Sex'], between=outcome)

        f.write('\nTract: {}'.format(tracts[i]))

        f.write('\nOutcome {}: \n'.format(outcome))
        f.write(ancova_age.to_string()) 
        f.write('\n- - - - - - - - - - - - - - - - - - - - - - - - - -')
        f.write('\nOutcome {}: \n'.format(outcome))
        f.write(ancova_agesex.to_string())

        f.write('\n___________________________________________________')
        f.write('\n')
    f.close()
    return filepath

In [ ]:
def getAllANOVAMeasures(datapd):
    for i in outcomes:
        for j in measures:
            getANOVA(datapd, j, i)

The following is to get T-Tests:

In [ ]:
def makeEmptyResultsDf():
    column_names = ['tract', 'outcome', 'measure', 'pvalue']
    results_df = pd.DataFrame(columns = column_names)
    return results_df

In [ ]:
def getTtest_file(datapd, measure=measures[0], outcome=outcomes[0]):
    filename = "{}_{}_agesexadjusted.txt".format(measure, outcome)
    filepath = os.path.join(ttestDir, filename)
    f = open(filepath, "w")
    for i in range(len(tracts)):
        tract_measures = getTractMeasure(datapd, outcome, tracts[i], measure)
        outcome0 = tract_measures.loc[tract_measures[outcome] == 0 ]
        outcome1 = tract_measures.loc[tract_measures[outcome] == 1 ]
        result = stats.ttest_ind(outcome0[measure.upper()], outcome1[measure.upper()])

        f.write('\nTract: {}'.format(tracts[i]))
        f.write('\nOutcome {}: \n'.format(outcome))
        f.write("Statistic: {}, P-value: {}".format(result.statistic, result.pvalue))
        f.write('\n___________________________________________________')
        f.write('\n')
    f.close()
    return filepath

In [ ]:
def getTtest_df(datapd, results_df, measure=measures[0], outcome=outcomes[0]):    
    for i in range(len(tracts)):
        tract_measures = getTractMeasure(datapd, outcome, tracts[i], measure)
        outcome0 = tract_measures.loc[tract_measures[outcome] == 0 ]
        outcome1 = tract_measures.loc[tract_measures[outcome] == 1 ]
        result = stats.ttest_ind(outcome0[measure.upper()], outcome1[measure.upper()])
        row = {'tract': tracts[i], 'outcome': outcome, 'measure': measure, 'pvalue': result.pvalue}
        results_df = results_df.append(row, ignore_index=True)
    return results_df

In [ ]:

def getAllTtests_df(datapd):
    results_df = makeEmptyResultsDf()
    for i in outcomes:
        for j in measures:
            results_df = getTtest_df(datapd, results_df, j, i)
    return results_df

In [ ]:
data = getData(datasrc)
#getAllANOVAMeasures(data)
results_df = getAllTtests_df(data)

In [ ]:
results_df_pcpc = results_df.loc[results_df['outcome'] == outcomes[0]]
results_wide = results_df_pcpc.pivot(index=['tract'], columns='measure', values='pvalue')
results_wide

In [ ]:
filename = "agesexadjusted_ttestresults.csv"
filepath = os.path.join(ttestDir, filename)
results_df.to_csv(filepath, index=False, sep=',')